In [1]:
from transformers import AutoModel, AutoTokenizer, AutoModelForTokenClassification

In [2]:
# checkpoint_path = '/home/ksaputa/mspace/plotkarzyna/models/herbert-large/chesckpoint-2080'
checkpoint_path = '/home/ksaputa/mspace/plotkarzyna/models/herbert-large-2/checkpoint-6240'
model = AutoModelForTokenClassification.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path, return_tensors='pt')

In [12]:
tokenizer("Ala ma kota, który wszedł na drzewo.")

{'input_ids': [0, 37, 2121, 2185, 24112, 1947, 2377, 12745, 1998, 16621, 1899, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [5]:
def decode_bioes_nested(labels, tokens):
    """
    Decodes a sequence of BIOES tags to extract mentions, including nested mentions.

    Parameters:
    - tags (list): List of predicted tags like ['B', 'I', 'E', ...]

    Returns:
    - mentions (list of tuples): List of mention spans as (start, end) 
    """
    
    mentions = []
    mentions_str = []
    stack = []

    for idx, tag in enumerate(labels):
        if tag == 'B':
            stack.append(idx)
        elif tag == 'E':
            if stack:
                start_idx = stack.pop()
                mentions.append((start_idx, idx))
        elif tag == 'S':
            mentions.append((idx, idx))
        elif tag == 'I':
            if not stack:  # 'I' without a preceding 'B'
                stack.append(idx)
                
    # Handle unmatched 'B' or 'I' tags
    while stack:
        start_idx = stack.pop(0)
        if not stack:  # If there's no further 'B' or 'I' tag
            mentions.append((start_idx, len(labels) - 1))
        else:
            next_start_idx = stack[0]
            mentions.append((start_idx, next_start_idx - 1))

    mentions = sorted(mentions, key=lambda x: x[0])  # Sort by start index
    mentions_str = [tokens[start:(end+1)] for start, end in mentions]
    return mentions_str

# Test the function
# tags = ["B", "B", "E", "E", "S", "B", "I", "I"]
# print(decode_bioes_nested(tags))  # Expected [(0, 3), (1, 2), (4, 4), (5, 7)]

In [6]:
from plotkarzyna.decode import *

In [15]:
text = "Ala ma kota, który wszedł na drzewo."
# text = "Pogoda pod koniec tygodnia nie będzie miała dla nas litości. W dalszym ciągu spodziewamy się upałów. Miejscami temperatura zdecydowanie przekroczy 30 st. C i tylko lokalnie orzeźwienie przyniosą burze."
text = "Koalicja Obywatelska już odkryła karty i wiemy, jakie nazwiska znajdą się na listach w poszczególnych okręgach podczas jesiennych wyborów do Sejmu. Teraz czas na Lewicę. Partia ujawnia swoich kandydatów."
text = """Jeżeli pole to jest polem centralnym, zależnym tylko od odległości cząstek, a niezależnym od kierunku w przestrzeni, to teoria kwantowa przewiduje, że stan o najniższej energii winien mieć zerowy moment pędu i symetrię sferyczną. Tymczasem pomiary rozkładu ładunku elektrycznego w deuteronie wskazują na wyraźne, jakkolwiek niewielkie odstępstwa od takiej kulistej symetrii. Co więcej, nie można było pogodzić własności magnetycznych deuteronu z własnościami magnetycznymi jego składników. Ponieważ spin deuteronu wynosi 1, więc połówkowe spiny neutronu i protonu muszą być w nim ustawione równolegle. Kierunki wektorów momentów magnetycznych związane są z kierunkami spinów, zatem moment magnetyczny deuteronu winien być sumą momentów protonu i neutronu. Zaobserwowane odstępstwa świadczą o tym, że pewien przyczynek do momentu magnetycznego deuteronu musi pochodzić z ruchu protonu po orbicie z momentem pędu większym od zera. Ponieważ w przypadku sił centralnych stan o najniższej energii musi mieć moment pędu równy zeru, możemy wyciągnąć stąd wniosek o niecentralności sił jądrowych. Muszą one, poza zależnością od odległości nukleonów, zależeć w jakiś sposób od kierunku w przestrzeni. Skąd jednak taka zależność może pochodzić?
 Oddziaływania między nukleonami mogą zależeć również od innych poza ich odległością cech nukleonów, takich jak pęd czy moment pędu w ich ruchu, a także ich spiny i izospiny. Jest oczywiste, że mając do dyspozycji obok wektora odległości R również wektory spinów nukleonów S1 i S2, możemy część niecentralną oddziaływania utworzyć jako zależną od wzajemnej orientacji tych wektorów (rys.4.4). Skąd jednak znaleźć możemy postać tej zależności?
"""

tokenized = tokenizer([text], return_tensors='pt')
pred = model(
    **tokenized
).logits.argmax(-1)[0]

print(' '.join([
    f"|{tokenizer.decode(tok, clean_up_tokenization_spaces=False)}| ({id2label[int(el)]})" for el, tok in zip(pred, list(tokenized['input_ids'][0]))
]))
tokens = [tokenizer.decode(tok) for tok in list(tokenized['input_ids'][0])]
labels = [id2label[int(el)] for el, tok in zip(pred, list(tokenized['input_ids'][0]))]
decode_bioes(labels, tokens)

|<s>| (O) |Jeżeli| (O) |pole| (B) |to| (E) |jest| (O) |polem| (B) |centralnym| (I) |,| (I) |zależ| (I) |nym| (I) |tylko| (I) |od| (I) |odległości| (B) |cząstek| (S) |,| (I) |a| (I) |niezależ| (I) |nym| (I) |od| (I) |kierunku| (B) |w| (I) |przestrzeni| (S) |,| (O) |to| (O) |teoria| (B) |kwan| (E) |towa| (E) |przewiduje| (O) |,| (O) |że| (O) |stan| (B) |o| (I) |najniż| (B) |szej| (B) |energii| (E) |winien| (O) |mieć| (O) |zer| (B) |owy| (I) |moment| (I) |pędu| (S) |i| (I) |sy| (B) |met| (I) |rię| (I) |s| (E) |fery| (E) |czną| (E) |.| (O) |Tymczasem| (O) |pomiary| (B) |rozkładu| (B) |ładunku| (B) |elektrycznego| (E) |w| (I) |de| (S) |u| (S) |tero| (S) |nie| (E) |wskazują| (O) |na| (O) |wyraźne| (B) |,| (I) |jakkolwiek| (I) |niewielkie| (I) |odstęp| (I) |stwa| (I) |od| (I) |takiej| (B) |ku| (I) |list| (I) |ej| (I) |sy| (E) |metrii| (E) |.| (O) |Co| (O) |więcej| (O) |,| (O) |nie| (O) |można| (O) |było| (O) |pogodzić| (O) |własności| (B) |magne| (I) |tycznych| (I) |de| (S) |u| (S) |te| (S) |

[['pole', 'to'],
 ['polem',
  'centralnym',
  ',',
  'zależ',
  'nym',
  'tylko',
  'od',
  'odległości',
  'cząstek',
  ',',
  'a',
  'niezależ',
  'nym',
  'od',
  'kierunku',
  'w',
  'przestrzeni'],
 ['odległości',
  'cząstek',
  ',',
  'a',
  'niezależ',
  'nym',
  'od',
  'kierunku',
  'w',
  'przestrzeni'],
 ['cząstek',
  ',',
  'a',
  'niezależ',
  'nym',
  'od',
  'kierunku',
  'w',
  'przestrzeni'],
 ['teoria', 'kwan', 'towa'],
 ['stan', 'o', 'najniż', 'szej', 'energii'],
 ['najniż', 'szej', 'energii'],
 ['szej', 'energii'],
 ['zer',
  'owy',
  'moment',
  'pędu',
  'i',
  'sy',
  'met',
  'rię',
  's',
  'fery',
  'czną'],
 ['pędu', 'i', 'sy', 'met', 'rię', 's', 'fery', 'czną'],
 ['ładunku', 'elektrycznego'],
 ['rozkładu', 'ładunku', 'elektrycznego', 'w', 'de', 'u', 'tero', 'nie'],
 ['pomiary',
  'rozkładu',
  'ładunku',
  'elektrycznego',
  'w',
  'de',
  'u',
  'tero',
  'nie'],
 ['de', 'u', 'tero', 'nie'],
 ['takiej', 'ku', 'list', 'ej', 'sy', 'metrii'],
 ['wyraźne',
  ',

In [8]:
decode_bioes(labels, tokens)

[['pole', 'to'],
 ['polem',
  'centralnym',
  ',',
  'zależ',
  'nym',
  'tylko',
  'od',
  'odległości',
  'cząstek',
  ',',
  'a',
  'niezależ',
  'nym',
  'od',
  'kierunku',
  'w',
  'przestrzeni'],
 ['odległości',
  'cząstek',
  ',',
  'a',
  'niezależ',
  'nym',
  'od',
  'kierunku',
  'w',
  'przestrzeni'],
 ['cząstek',
  ',',
  'a',
  'niezależ',
  'nym',
  'od',
  'kierunku',
  'w',
  'przestrzeni'],
 ['teoria', 'kwan', 'towa'],
 ['stan', 'o', 'najniż', 'szej', 'energii'],
 ['najniż', 'szej', 'energii'],
 ['szej', 'energii'],
 ['zer',
  'owy',
  'moment',
  'pędu',
  'i',
  'sy',
  'met',
  'rię',
  's',
  'fery',
  'czną'],
 ['pędu', 'i', 'sy', 'met', 'rię', 's', 'fery', 'czną'],
 ['ładunku', 'elektrycznego'],
 ['rozkładu', 'ładunku', 'elektrycznego', 'w', 'de', 'u', 'tero', 'nie'],
 ['pomiary',
  'rozkładu',
  'ładunku',
  'elektrycznego',
  'w',
  'de',
  'u',
  'tero',
  'nie'],
 ['de', 'u', 'tero', 'nie'],
 ['takiej', 'ku', 'list', 'ej', 'sy', 'metrii'],
 ['wyraźne',
  ',

In [9]:
decode_bioes_nested(labels, tokens)

[['pole', 'to'],
 ['polem', 'centralnym', ',', 'zależ', 'nym', 'tylko', 'od'],
 ['odległości',
  'cząstek',
  ',',
  'a',
  'niezależ',
  'nym',
  'od',
  'kierunku',
  'w',
  'przestrzeni',
  ',',
  'to',
  'teoria',
  'kwan',
  'towa',
  'przewiduje',
  ',',
  'że'],
 ['cząstek'],
 ['kierunku', 'w', 'przestrzeni', ',', 'to', 'teoria', 'kwan', 'towa'],
 ['przestrzeni'],
 ['teoria', 'kwan'],
 ['stan',
  'o',
  'najniż',
  'szej',
  'energii',
  'winien',
  'mieć',
  'zer',
  'owy',
  'moment',
  'pędu',
  'i',
  'sy',
  'met',
  'rię',
  's',
  'fery',
  'czną',
  '.',
  'Tymczasem'],
 ['najniż',
  'szej',
  'energii',
  'winien',
  'mieć',
  'zer',
  'owy',
  'moment',
  'pędu',
  'i',
  'sy',
  'met',
  'rię',
  's',
  'fery',
  'czną'],
 ['szej', 'energii'],
 ['zer', 'owy', 'moment', 'pędu', 'i', 'sy', 'met', 'rię', 's', 'fery'],
 ['pędu'],
 ['sy', 'met', 'rię', 's'],
 ['pomiary',
  'rozkładu',
  'ładunku',
  'elektrycznego',
  'w',
  'de',
  'u',
  'tero',
  'nie',
  'wskazują',
  

In [10]:
labels

['O',
 'O',
 'B',
 'E',
 'O',
 'B',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'B',
 'S',
 'I',
 'I',
 'I',
 'I',
 'I',
 'B',
 'I',
 'S',
 'O',
 'O',
 'B',
 'E',
 'E',
 'O',
 'O',
 'O',
 'B',
 'I',
 'B',
 'B',
 'E',
 'O',
 'O',
 'B',
 'I',
 'I',
 'S',
 'I',
 'B',
 'I',
 'I',
 'E',
 'E',
 'E',
 'O',
 'O',
 'B',
 'B',
 'B',
 'E',
 'I',
 'S',
 'S',
 'S',
 'E',
 'O',
 'O',
 'B',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'B',
 'I',
 'I',
 'I',
 'E',
 'E',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'I',
 'I',
 'S',
 'S',
 'S',
 'S',
 'O',
 'B',
 'B',
 'I',
 'I',
 'B',
 'E',
 'O',
 'O',
 'B',
 'B',
 'S',
 'S',
 'S',
 'S',
 'O',
 'S',
 'O',
 'O',
 'B',
 'I',
 'B',
 'I',
 'I',
 'B',
 'S',
 'B',
 'I',
 'S',
 'S',
 'O',
 'O',
 'O',
 'S',
 'O',
 'O',
 'O',
 'B',
 'B',
 'B',
 'B',
 'B',
 'I',
 'E',
 'E',
 'O',
 'O',
 'O',
 'B',
 'B',
 'S',
 'S',
 'O',
 'O',
 'B',
 'I',
 'I',
 'S',
 'S',
 'S',
 'S',
 'O',
 'O',
 'B',
 'O',
 'B',
 'B',
 'B',
 'S',
 'I',
 'S',
 'S',
 'S',
 'O',
 'B',
 'B',
 'B',
 'E'

In [11]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), 